# Generative Adversarial Networks (GANs)

This notebook is a Pytorch implementation of the GAN architecture proposed in the [paper](https://arxiv.org/pdf/1406.2661) by Goodfellow et. al. 

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 100
noise_dim = 100
batch_size = 64
learning_rate = 0.0002
beta1 = 0.5

In [12]:
class G(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(G, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),    
            nn.ReLU(),

            nn.Linear(256, 512),    
            nn.ReLU(),

            nn.Linear(512, output_dim),
            nn.Tanh()                
        )

    def forward(self, z):
        return self.model(z)


In [13]:
class D(nn.Module):
    def __init__(self, input_dim):
        super(D, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256), # fc layer
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.model(x)

In [14]:
criterion = nn.BCELoss() # the paper specifies the Binary Cross Entropy loss to be used and hence we set that up

# we shall the use the MNIST handwritten digits dataset for training the GAN, and we know the dimensions of the image of the MNIST dataset is 28x28, which is 784 pixels in total
# hence we set the input and output dimensions of the generator and discriminator accordingly

G_model = G(input_dim=100, output_dim=784).to(device)  # Example: 100-dim noise, 28x28 image
D_model = D(input_dim=784).to(device)  # Example: 28x28 image flattened to 784

lr = 0.0002 # learning rate for both generator and discriminator

# we shall use the adam optimizer for both the generator and discriminator, as it is a popular choice for training GANs
G_optimizer = torch.optim.Adam(G_model.parameters(), lr=lr)
D_optimizer = torch.optim.Adam(D_model.parameters(), lr=lr)

In [15]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split


transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


full_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)


train_size = int(0.9 * len(full_dataset))  # e.g., 90% for training
test_size = len(full_dataset) - train_size  # 10% for test/eval


train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [16]:
import os
import matplotlib.pyplot as plt
import torchvision.utils as vutils

# Make directory for saving generated images
os.makedirs("generated_images", exist_ok=True)

# Fixed noise to track progress across epochs
fixed_noise = torch.randn(64, noise_dim).to(device)

def show_and_save_generated_images(images, epoch):
    # Reshape and normalize
    grid = vutils.make_grid(images.view(-1, 1, 28, 28), normalize=True, nrow=8)
    
    # Plot
    plt.figure(figsize=(6, 6))
    plt.axis("off")
    plt.title(f"Generated Images - Epoch {epoch+1}")
    plt.imshow(grid.permute(1, 2, 0).cpu().numpy())
    
    # Save to file
    filename = f"generated_images/epoch_{epoch+1:03d}.png"
    plt.savefig(filename, bbox_inches='tight')
    plt.close()

for epoch in range(num_epochs):
    for batch_idx, (real_images, _) in enumerate(train_loader):
        real_images = real_images.view(real_images.size(0), -1)
        # Add Gaussian noise (helps D generalize)
        real_images += 0.05 * torch.randn_like(real_images)
        real_images = real_images.to(device)

        real_labels = (torch.ones(real_images.size(0), 1) * 0.9).to(device)  # label smoothing
        fake_labels = torch.zeros(real_images.size(0), 1).to(device)

        # Train Discriminator
        z = torch.randn(real_images.size(0), noise_dim).to(device)
        fake_images = G_model(z)

        real_outputs = D_model(real_images)
        fake_outputs = D_model(fake_images.detach())

        d_loss_real = criterion(real_outputs, real_labels)
        d_loss_fake = criterion(fake_outputs, fake_labels)
        d_loss = d_loss_real + d_loss_fake

        D_optimizer.zero_grad()
        d_loss.backward()
        D_optimizer.step()

        # Train Generator
        z = torch.randn(real_images.size(0), noise_dim).to(device)
        fake_images = G_model(z)
        fake_outputs = D_model(fake_images)

        g_loss = criterion(fake_outputs, real_labels)

        G_optimizer.zero_grad()
        g_loss.backward()
        G_optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}] | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")

    # Generate & save images with fixed noise
    with torch.no_grad():
        fake_images = G_model(fixed_noise)
    show_and_save_generated_images(fake_images, epoch)

print("Training complete.")


Epoch [1/100] | D Loss: 0.3757 | G Loss: 3.2945
Epoch [2/100] | D Loss: 0.4758 | G Loss: 3.1871
Epoch [3/100] | D Loss: 0.6277 | G Loss: 2.0811
Epoch [4/100] | D Loss: 0.7854 | G Loss: 2.2312
Epoch [5/100] | D Loss: 0.6549 | G Loss: 1.9531
Epoch [6/100] | D Loss: 0.5383 | G Loss: 2.3253
Epoch [7/100] | D Loss: 0.5024 | G Loss: 2.8170
Epoch [8/100] | D Loss: 0.5260 | G Loss: 2.5227
Epoch [9/100] | D Loss: 0.5878 | G Loss: 2.7396
Epoch [10/100] | D Loss: 0.5252 | G Loss: 2.4311
Epoch [11/100] | D Loss: 0.5736 | G Loss: 2.7149
Epoch [12/100] | D Loss: 0.6390 | G Loss: 2.4957
Epoch [13/100] | D Loss: 0.6162 | G Loss: 2.6829
Epoch [14/100] | D Loss: 0.5744 | G Loss: 2.7442
Epoch [15/100] | D Loss: 0.5828 | G Loss: 2.5803
Epoch [16/100] | D Loss: 0.6027 | G Loss: 2.7863
Epoch [17/100] | D Loss: 0.6306 | G Loss: 3.0010
Epoch [18/100] | D Loss: 0.5714 | G Loss: 2.6523
Epoch [19/100] | D Loss: 0.4654 | G Loss: 3.2270
Epoch [20/100] | D Loss: 0.6557 | G Loss: 2.6001
Epoch [21/100] | D Loss: 0.68